# Acquirium - WaterTap - MQTT Integration

## Introduction

We built a single pump simulation using WaterTap. The pump model requires six input variables. Two of these variables, pump efficiency and pump pressure delta, are fixed. The remaining four variables are simulated as if they were produced by real sensors and are continuously published to MQTT.

We implemented a script that subscribes to MQTT and uses the incoming values as inputs to the WaterTap model. Since all six required variables are provided, the degrees of freedom of the model are reduced to zero. As a result, the model operates as a simulation rather than an optimization problem.

After running the simulation, we extract the WaterTap computed pump work and publish it back to MQTT. This output behaves like a soft sensor, where the value is derived from a physics based model rather than a physical measurement.

Below is an overview of the system architecture:
<div style="text-align: center;">

![overview](./figs/watertap_process.png)

</div>

We then modeled this system using a knowledge graph based on the WaTr ontology. The knowledge graph captures the structure of the pump, its associated variables, and the relationships between physical components, sensor streams, and computed quantities. This semantic representation allows us to reason over both the system topology and the available data streams, enabling flexible discovery, querying, and integration of sensor measurements and model derived values.

<div style="text-align: center;">

![overview](./figs/kg_model.png)

</div>

The properties store all the information required for data retrieval, along with additional metadata that describes the values produced by each sensor. This metadata specifies details such as the physical meaning of the measurement, its unit, the medium or substance being measured, and whether the value originates from a physical sensor or a model derived computation.

<div style="text-align: center;">

![overview](./figs/property.png)

</div>

Given a knowledge graph model, Acquirium can connect to MQTT and subscribe to the corresponding data streams. From Acquirium’s perspective, it does not matter whether the values are simulated or produced by real sensors; it simply consumes the data published on MQTT. In addition to MQTT, Acquirium can also connect to other data sources such as files, databases, and external services, providing a unified interface for accessing heterogeneous data streams. 

Acquirium stores both the data and its associated metadata in its own database, enabling efficient access, querying, and reuse in future analyses and applications.

<div style="text-align: center;">

![overview](./figs/acquirium_pull.png)

</div>

## Acquirium Code Example:

Now we'll show how we can retrieve data using acquirium:

#### Initial Steps

In [ ]:
# Package imports

from datetime import datetime, timedelta
from acquirium import Acquirium
from acquirium.Client.query import Query
from acquirium.internals.internals_namespaces import (
    HAS_UNIT,
    HAS_MEDIUM,
    HAS_QUANTITY_KIND,
    OF_SUBSTANCE,
    S223,
    WATR,
    UNIT
)
def banner(msg: str) -> None:
    print("\n" + "=" * 80)
    print(msg)
    print("=" * 80 + "\n")


In [ ]:
## Initial setup: create Acquirium instance
banner("1) Create Acquirium session")
acq = Acquirium(
        server_url="localhost",
        server_port=8000,
        use_ssl=False,
        lexicon_path="../ontologies/lexicon.json",
    )

In [ ]:
## Add a graph to acquirium
## This will automatically connect the data sources to database and ingest timeseries data 
## Since our data is MQTT Streams, acquirium will subscribe to those streams and start pulling data
banner("2) Add a graph to acquirium")
acq.insert_graph("../deployments/WATERTAP/models/watertap-simple-pipe-model-with-ext-refs.ttl")


#### Querying

In [ ]:
banner("3) Start a new query and add an entity node")

#create a query object
# text matching to find ontology classes
q1 : Query = acq.find_entity(_class="inlet connection pt", alias="pump-in")
q1.show_query_graph() # show current query graph
q1.metadata_head() # show metadata of the current query graph nodes

In [ ]:
# Find a related node to our first node
q1 : Query = q1.find_related(_class="pump", alias="pump", _from="pump-in", hops=1)
q1.show_query_graph()
q1.metadata_head()

#### Data Retrieval

In [ ]:
q2 :Query = acq.find_all_data()
q2.show_query_graph()
q2.metadata_head()
a = q2.data_head()

#### Apply Filters:

In [ ]:
q3 : Query = q2.filter_by_unit(unit = "kilogram per second")
q3.show_query_graph()
q3.metadata_head()
b = q3.data_head()

In [ ]:
from IPython.display import clear_output

import time
while True:
    a = q2.latest_data()
    print(a)
    time.sleep(5)
    clear_output(wait=True)